In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
import pytesseract
import pyodbc
import re

In [ ]:
sql_conn = pyodbc.connect('DRIVER={SQL Server};'
                            'SERVER=L_AAGDATIX;'
                            'DATABASE=DatixCRM;'
                            'Trusted_Connection=yes') 
query = "set transaction isolation level read uncommitted select b.description from DatixCRM.dbo.code_locactual b join DatixCRM.dbo.incidents_main a on a.inc_locactual = b.code where a.inc_type='PAT' and a.inc_category='MEDIC' and a.inc_organisation='QA' and b.cod_parent2='QA'"
df = pd.read_sql(query, sql_conn)
df.columns = ['location']
df.head()

In [ ]:
def find_ward(location):
    ward = re.findall(r"[A-G][0-9]",location)
    if len(ward)>0: return ward[0]
    else: return location

In [ ]:
def find_ward1to4(location):
    ward1to4 = re.findall(r"[D-G][1-4]",location)
    if len(ward1to4)>0: return ward1to4[0][0]+'3'
    else: return location

In [ ]:
def find_wardBCF5to7(location):
    ward5to7 = re.findall(r"(B|C|F)[5-7]",location)
    if len(ward5to7)>0: return ward5to7[0][0]+'6'
    else: return location

In [ ]:
def find_wardEG6to8(location):
    ward6to8 = re.findall(r"(E|G)[6-8]",location)
    if len(ward6to8)>0: return ward6to8[0][0]+'7'
    else: return location

In [ ]:
def find_acronym(location):
    acronym = re.findall(r"\(([A-Z]+)\)",location)
    if len(acronym)>0: return acronym[0]
    else: return location

In [ ]:
def find_haematology_or_oncology(location):
    if 'Haematology' in location or 'Oncology' in location: return 'Haematology & Oncology Centre'
    else: return location

In [ ]:
def find_ED(location):
    if 'Emergency Department' in location: return 'Emergency Department'
    else: return location

In [ ]:
def find_Ophthalmology(location):
    if 'Ophthalmology' in location: return 'Eye'
    else: return location

In [ ]:
def find_surgical(location):
    if 'Surgical' in location: return 'E3'
    else: return location

In [ ]:
df['location'] = np.vectorize(find_ward)(df['location'])
df['location'] = np.vectorize(find_ward1to4)(df['location'])
df['location'] = np.vectorize(find_wardBCF5to7)(df['location'])
df['location'] = np.vectorize(find_wardEG6to8)(df['location'])
#df['location'] = np.vectorize(find_acronym)(df['location'])
df['location'] = np.vectorize(find_haematology_or_oncology)(df['location'])
df['location'] = np.vectorize(find_ED)(df['location'])
df['location'] = np.vectorize(find_Ophthalmology)(df['location'])
df['location'] = np.vectorize(find_surgical)(df['location'])

In [ ]:
df.loc[df['location'] == 'A7', 'location'] = 'Paediatric Unit'
df.loc[df['location'] == 'A8', 'location'] = 'Paediatric Unit'
df.loc[df['location'] == 'Administration Offices Renal & Transplant', 'location'] = 'Renal Unit'
df.loc[df['location'] == 'Renal Day Unit  QA', 'location'] = 'Renal Unit'
df.loc[df['location'] == 'Anti Coagulation Clinic QA', 'location'] = 'Haematology & Oncology Centre'
df.loc[df['location'] == 'Cardiac Catheter Laboratory', 'location'] = 'Cardiology'
df.loc[df['location'] == 'Cardiac Day Unit (CDU)', 'location'] = 'Cardiology'
df.loc[df['location'] == 'Cardiology Outpatients QA', 'location'] = 'C Level Outpatients'
df.loc[df['location'] == 'Childrens Assesment Unit (CAU)', 'location'] = 'Paediatric Unit'
df.loc[df['location'] == 'Colorectal Outpatients QA', 'location'] = 'C Level Outpatients'
df.loc[df['location'] == 'Diabetes and Endocrinology Centre QA', 'location'] = 'Diabetes and Endocrinology'
df.loc[df['location'] == 'ENT Outpatients QA', 'location'] = 'Ear, Nose & Throat'
df.loc[df['location'] == 'Early Pregnancy Assessment Unit', 'location'] = 'A5'
df.loc[df['location'] == 'Endoscopy Unit D Level QA', 'location'] = 'Endoscopy Unit'
df.loc[df['location'] == 'Eye Day Case Unit QA', 'location'] = 'Eye Unit'
df.loc[df['location'] == 'Gastroenterology  Outpatients QA', 'location'] = 'D Level Outpatients'
df.loc[df['location'] == 'Gynae Outpatients QA', 'location'] = 'Gynaecology Outpatients'
df.loc[df['location'] == 'Haemodialysis Unit', 'location'] = 'Renal Haemodialysis'
df.loc[df['location'] == 'Maxillofacial Outpatients QA', 'location'] = 'D Level Outpatients'
df.loc[df['location'] == 'Ophthalmology Eye Emergency Department', 'location'] = 'Eye Unit'
df.loc[df['location'] == 'Pathology Department', 'location'] = 'Pathology'
df.loc[df['location'] == 'Peritoneal Dialysis', 'location'] = 'Renal Unit'
df.loc[df['location'] == 'Radiotherapy  Dept within CHOC', 'location'] = 'F6'
df.loc[df['location'] == 'Receipt and Delivery', 'location'] = 'Reception'
df.loc[df['location'] == 'Reception (Atrium)', 'location'] = 'Reception'
df['location'] = df['location'].str.replace(' Unit','')
df.loc[df['location'] == 'Renal', 'location'] = 'Renal Unit'

In [ ]:
df = df[df['location'] != "Administration Offices Women and Children's"]
df = df[df['location'] != 'Anaesthetics Department'] 
df = df[df['location'] != 'Antenatal Clinic'] 
df = df[df['location'] != 'Diagnostic Imaging'] 
df = df[df['location'] != 'ICT  Centre QA'] 
df = df[~df['location'].str.contains('Maternity')] 
df = df[df['location'] != 'Observation Ward']
df = df[df['location'] != 'Occupational Health']
df = df[df['location'] != 'Orthopaedic Pre Operation Clinic QA']
df = df[df['location'] != 'Pre-Operative Assessment (POA)']
df = df[~df['location'].str.contains('QA@ Home ')] 
df = df[df['location'] != 'Research & Development Department']
df = df[df['location'] != 'Respiratory Day']
df = df[df['location'] != 'TIA Clinic']
df = df[df['location'] != 'Theatres QA']
df = df[df['location'] != 'Transplant and Retrieval  QA']

In [ ]:
location_counts = df['location'].value_counts().sort_index(ascending=True)
print(location_counts)

In [ ]:
df_counts = location_counts.rename_axis('location').reset_index(name='events')
print(df_counts)

In [ ]:
df_counts['floor'] = df_counts['location'].str[0]
df_counts.loc[df_counts['location'] == 'Acute Medical (AMU) (MAU)', 'floor'] = 'C'
df_counts.loc[df_counts['location'] == 'Diabetes and Endocrinology', 'floor'] = 'C'
df_counts.loc[df_counts['location'] == 'Discharge', 'floor'] = 'A'
df_counts.loc[df_counts['location'] == 'Ear, Nose & Throat', 'floor'] = 'D'
df_counts.loc[df_counts['location'] == 'Emergency Department', 'floor'] = 'C'
df_counts.loc[df_counts['location'] == 'Endoscopy', 'floor'] = 'D'
df_counts.loc[df_counts['location'] == 'Eye', 'floor'] = 'B'
df_counts.loc[df_counts['location'] == 'Gynaecology Outpatients', 'floor'] = 'D'
df_counts.loc[df_counts['location'] == 'Haematology & Oncology Centre', 'floor'] = 'B'
df_counts.loc[df_counts['location'] == 'Medical Physics Department', 'floor'] = 'F'
df_counts.loc[df_counts['location'] == 'Paediatric', 'floor'] = 'A'
df_counts.loc[df_counts['location'] == 'Pathology', 'floor'] = 'E'
df_counts.loc[df_counts['location'] == 'Pharmacy QA', 'floor'] = 'C'
df_counts.loc[df_counts['location'] == 'Reception', 'floor'] = 'A'
df_counts.loc[df_counts['location'] == 'Rheumatology Outpatients QA', 'floor'] = 'C'
df_counts.loc[df_counts['location'] == 'Theatre Admissions Suite (TAS)', 'floor'] = 'E'

In [ ]:
def find_renal(location,floor):
    if 'Renal' in location: return 'G'
    else: return floor

In [ ]:
df_counts['floor'] = np.vectorize(find_renal)(df_counts['location'],df_counts['floor'])

In [ ]:
df_counts['location'] = df_counts['location'].str.replace(' QA','')
print(df_counts.sort_values(by=['floor']))

In [ ]:
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
text_dict = {}
floors = ['A','B','C','D','E','F','G']
for floor in floors:
    text = pytesseract.image_to_data(Image.open('level_maps_'+floor.lower()+'.png'),output_type='data.frame')
    text = text.dropna()
    text = text[text.text != ' ']
    text = text[text.text != '  ']
    text = text[text.text != '   ']
    text = text[text.text != '    ']
    text = text[text.text != '     ']
    text_dict[floor] = text
print(text_dict)

In [ ]:
text_dict['A'].loc[text_dict['A'].text == 'AS,', 'text'] = 'A5'
text_dict['A'].loc[text_dict['A'].text == 'Dischage', 'text'] = 'Discharge'
text_dict['B'].loc[text_dict['B'].text == 'Centre', 'text'] = 'Haematology & Oncology Centre'
text_dict['B'].loc[text_dict['B'].text == '(B8)', 'text'] = 'B8'
text_dict['B'].loc[text_dict['B'].text == 'Neonatal', 'text'] = 'B9'
text_dict['B'].loc[text_dict['B'].text == 'Eye', 'left'] = 1100
text_dict['C'].loc[text_dict['C'].text == 'Day', 'text'] = 'Acute Medical (AMU) (MAU)'
text_dict['C'].loc[text_dict['C'].text == 'Acute Medical (AMU) (MAU)', 'top'] = 354
text_dict['C'].loc[text_dict['C'].text == 'Ambulance', 'left'] = 700
text_dict['C'].loc[text_dict['C'].text == 'Ambulance', 'text'] = 'C Level Outpatients'
text_dict['C'].loc[text_dict['C'].text == 'C Level Outpatients', 'left'] = 50
text_dict['C'].loc[text_dict['C'].text == 'Diabetes', 'text'] = 'Diabetes and Endocrinology'
text_dict['C'].loc[text_dict['C'].text == 'Blood', 'text'] = 'Emergency Department'
text_dict['C'].loc[text_dict['C'].text == 'Emergency Department', 'left'] = 1100
text_dict['C'].loc[text_dict['C'].text == 'Rheumatology', 'text'] = 'Rheumatology Outpatients'
text_dict['D'].loc[text_dict['D'].text == 'Day', 'text'] = 'Day Surgery'
text_dict['D'].loc[text_dict['D'].text == 'Nose', 'text'] = 'Ear, Nose & Throat'
text_dict['D'].loc[text_dict['D'].text == 'Level', 'text'] = 'D Level Outpatients'
text_dict['D'].loc[text_dict['D'].text == 'Gynaecology', 'text'] = 'Gynaecology Outpatients'
text_dict['E'].loc[text_dict['E'].text == 'Theatre', 'text'] = 'Theatre Admissions Suite (TAS)'
text_dict['E'].loc[text_dict['E'].text == 'E5)', 'text'] = 'E5'
text_dict['F'].loc[text_dict['F'].text == 'Medical', 'text'] = 'Medical Physics Department'
text_dict['G'].loc[text_dict['G'].text == 'Unit', 'text'] = 'Renal Unit'
text_dict['G'].loc[text_dict['G'].text == 'G8,', 'text'] = 'G5'
text_dict['G'].loc[text_dict['G'].text == 'G5', 'top'] = 750
text_dict['G'].loc[text_dict['G'].text == 'G5', 'left'] = 700
text_dict['G'].loc[text_dict['G'].text == 'G2,', 'text'] = 'G3'
text_dict['G'].loc[text_dict['G'].text == 'Haemodialysis', 'text'] = 'Renal Haemodialysis'
text_dict['G'].loc[text_dict['G'].text == 'Outpatients', 'text'] = 'Renal Outpatients'

In [ ]:
merged_dict = {}
for floor in floors:
    merged_dict[floor] = pd.merge(left=df_counts,right=text_dict[floor],left_on='location',right_on='text')
print(merged_dict)

In [ ]:
cm = plt.cm.get_cmap('RdYlGn_r')
vmin = df_counts['events'].min()
vmax = df_counts['events'].max()
area_multiplication = 6

In [ ]:
for floor in floors:
    plt.figure()
    img=mpimg.imread('level_maps_'+floor.lower()+'.png')
    plt.figure(figsize=(60,6))
    imgplot = plt.imshow(img)
    fig = plt.gca()
    x = merged_dict[floor].left.values + merged_dict[floor].width.values/2
    y = merged_dict[floor].top.values + merged_dict[floor].height.values/2
    print(merged_dict[floor].events.values)
    size = merged_dict[floor].events.values*area_multiplication
    print(size)
    color = merged_dict[floor].events.values
    print(color)
    plt.scatter(x,y,s=size,c=color,vmin=vmin,vmax=vmax,cmap=cm,alpha=0.5)
    plt.colorbar(label='# of medicine patient safety events')
    fig.get_xaxis().set_visible(False)
    fig.get_yaxis().set_visible(False)